In [5]:
from pymarc import Record, Field
from pymarc import MARCWriter
import openpyxl

In [6]:
import csv
from dataclasses import dataclass
import datetime
import re
import os

In [7]:
# Merge the exported DigitalCommons spreadsheets

def parse_workbook(file):
    print(file)
    wb = openpyxl.load_workbook(file)
    ws = wb.active #to select the activated worksheets
    # current_sheet = wb['1']
    sheet_dict = dict()
    for num, row in enumerate(ws.iter_rows()):
        if num == 0:
            keys = [i.value or num for i in row]
            continue
        values = [i.value for i in row]
        row_dict = {keys[i]: values[i] for i in range(len(keys))}
        sheet_dict[num] = row_dict
    return sheet_dict

def merge_dicts(*dicts):
    counter = 0
    merged_dict = dict()
    for d in dicts:
        for _, i in d.items():
            try:
                itemid = i['itemid']
            except KeyError:
                itemid = i['urn']
            if not itemid:
                merged_dict[counter] = i
                counter += 1
            elif itemid in merged_dict:
                print(i)
                print('duplicate {}'.format(itemid))
                break
            else:
                merged_dict[itemid] = i
    return merged_dict

# these filname and paths will change depending on where the input files are named and located
thesis_file = 'gradschool_theses_Spring2020_to_June2022.xlsx'
dissertation_file = "gradschool_dissertations_Spring2020_to_June2022.xlsx"

digcomm_diss = parse_workbook(dissertation_file)
# digcomm_historical = parse_workbook()
# digcomm_majorp = parse_workbook()
digcomm_thess = parse_workbook(thesis_file)
merged_digcomm = merge_dicts(digcomm_thess, digcomm_diss )


gradschool_dissertations_Spring2020_to_June2022.xlsx
gradschool_theses_Spring2020_to_June2022.xlsx


In [8]:
def merged_dict_to_csv(output_filename, source_dict):
    with open(output_filename, 'w', encoding='utf-8', newline='') as f:
        for urn, item_dict in source_dict.items():
            headers = sorted(item_dict.keys())
            break
        w = csv.DictWriter(f, headers)
        w.writeheader()
        for urn, item_dict in source_dict.items():
            w.writerow(item_dict)

# # the merge filename and path will change depending on where it should land
merge_file = 'DigCommPossiblyNotInCatalog.csv'
merged_dict_to_csv(merge_file, merged_digcomm)

In [9]:
@dataclass
class EtdItem:
    abstract: str
    advisor1: str
    advisor1_email: str
    advisor1_title: str
    advisor2: str
    advisor2_email: str
    advisor2_title: str
    advisor3: str
    advisor3_email: str
    advisor3_title: str
    advisor4: str
    advisor4_email: str
    advisor4_title: str
    advisor5: str
    advisor5_email: str
    advisor5_title: str
    advisor6: str
    advisor6_email: str
    advisor6_title: str
    advisor7: str
    advisor7_email: str
    advisor7_title: str
    author1_email: str
    author1_fname: str
    author1_institution: str
    author1_lname: str
    author1_mname: str
    author1_suffix: str
    author2_email: str
    author2_fname: str
    author2_institution: str
    author2_lname: str
    author2_mname: str
    author2_suffix: str
    author3_email: str
    author3_fname: str
    author3_institution: str
    author3_lname: str
    author3_mname: str
    author3_suffix: str
    author4_email: str
    author4_fname: str
    author4_institution: str
    author4_lname: str
    author4_mname: str
    author4_suffix: str
    author5_email: str
    author5_fname: str
    author5_institution: str
    author5_lname: str
    author5_mname: str
    author5_suffix: str
    availability: str
    availability_description: str
    calc_url: str
    context_key: str
    ctmtime: str
    defense_date: str
    degree_application: str
    degree_name: str
    department: str
    disciplines: str
    document_type: str
    embargo_date: str
    file_name: str
    file_size: str
    fulltext_url: str
    hide_author_email: str
    issue: str
    keywords: str
    legacy_department: str
    lsu_author_email: str
    publication_date: str
    release_date: str
    season: str
    semester_graduation: str
    submission_date: str
    title: str
    urn: str
    doi: str


In [10]:
# this is the same file as the merge file
source_csv = 'DigCommPossiblyNotInCatalog.csv'

rows_dataclass = []
with open(source_csv, newline='', encoding='utf-8') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=",", quotechar='"')
    for number, row in enumerate(spamreader):
        if number == 0:
            headers = row
            continue
    
        row_dict = dict(zip(headers, row))
        row_dataclass = EtdItem(**row_dict)
        rows_dataclass.append(row_dict)
#rows_dataclass is list of dictionaries:
    # print(rows_dataclass[0]["author1_fname"])

In [23]:
def join_name_parts(first, middle, last, suffix):
    name_str = ' '.join(i.strip() for i in (first, middle, last) if i)
    if suffix.strip():
        name_str += ', {}.'.format(suffix.strip().replace('.', ''))
    else:
        name_str += '.'
    return name_str

def reverse_name_parts(first, middle, last):
    name_strg = f"{last}, {' '.join(i for i in (first, middle) if i) }"
    return name_strg

def year_only(publication_date):
    year = publication_date.split('-')[0]
    return year

def degree_abbr(full_degree):
    match = re.search(r'\([A-Za-z]+\)', full_degree)
    if match:
        return match.group(0).replace('(', '').replace(')', '')
    return ''

def make_keyword_subfields(keywords):
    empty_list = []
    for i in keywords.split(', '):
        empty_list.append('a')
        empty_list.append(i)
    return empty_list

def comma_if_present(text):
    if text:
        return f'{ text },'
    return ''

In [24]:
def marc_template(row_dataclass):
    formal_name = reverse_name_parts(row_dataclass["author1_fname"],
                                     row_dataclass["author1_mname"],
                                     row_dataclass["author1_lname"])
    
    #publication_date = year_only(row_dataclass.publication_date)
    publication_date = year_only(row_dataclass["submission_date"])

    print(publication_date)
    
    if 'thesis' in row_dataclass["document_type"]:
        document_type_normal = 'Thesis'
        name_959a = 'ELECTRONIC THESES'
        name_959x = 'E-THESIS'
    elif 'dissertation' in row_dataclass["document_type"]:
        document_type_normal = 'Dissertation'
        name_959a = 'ELECTRONIC DISSERTATION'
        name_959x = 'E-DISS'
    else:
        print(f'no match for item {row_dataclass.urn}')
        return
        
        
    leader_text = "01401nam a2200313Ki 4500"
    record = Record(force_utf8=True, leader=leader_text)
    
    record.add_field(
        Field(
            tag = '008',
            data = f'{datetime.date.today():%y%m%d}s{publication_date}    laua    sbm   000 0 eng d'
        ))
    
    record.add_field(        
        Field(
            tag = '040',
            indicators = [' ', ' '],
            subfields = [
                'a', "LUU",
                'b', "eng",
                'e', "rda",
                'c', "LUU",
            ]))
    record.add_field(     
        Field(
            tag = '090',
            indicators = [' ', ' '],
            subfields = [
                'b', '',
            ]))
    record.add_field(
        Field(
            tag = '049',
            indicators = [' ', ' '],
            subfields = [
                'a', 'LUUU',
            ]))
    record.add_field(
        Field(
            tag = '100',
            indicators = ['1', ' '],
            subfields = [   
                'a', f'{ formal_name },',
                'c', f'{comma_if_present(row_dataclass["author1_suffix"])}',
                'e', 'author.'
            ]))
    # putting logic in the template, i'm ashamed.
    if len(row_dataclass["title"].split(':')) == 1:
        record.add_field(
            Field(
                tag = '245',
                indicators = ['1','0'],
                subfields = [
                    'a', f'{ row_dataclass["title"] } /',
                    'c', join_name_parts(row_dataclass["author1_fname"],
                                        row_dataclass["author1_mname"],
                                        row_dataclass["author1_lname"],
                                        row_dataclass["author1_suffix"])
                ]))
    else:
        record.add_field(
            Field(
                tag = '245',
                indicators = ['1','0'],
                subfields = [
                    'a', f'{row_dataclass["title"].split(":")[0]} :',
                    'b', f"""{":".join(row_dataclass["title"].split(":")[1:])} /""",
                    'c', f'by {join_name_parts(row_dataclass["author1_fname"],row_dataclass["author1_mname"],row_dataclass["author1_lname"],row_dataclass["author1_suffix"])}'
                ]))
    record.add_field(
        Field(
            tag = '264',
            indicators = [' ', '1'],
            subfields = [
                'a', "[Baton Rouge, Louisiana] :",
                'b', "[Louisiana State University],",
                'c', f'[{ publication_date }]',
            ]))
    record.add_field(
        Field(
            tag = '300',
            indicators = [' ', ' '],
            subfields = [
                'a', "1 online resource (leaves) :",
                'b', "illustrations, text file, PDF",
            ]))
    record.add_field(
        Field(
            tag = '336',
            indicators = [' ', ' '],
            subfields = [
                'a', 'text',
                'b', 'txt',
                '2', 'rdacontent',
            ]))
    record.add_field(
        Field(
            tag = '337',
            indicators = [' ', ' '],
            subfields = [
                'a', 'computer',
                'b', 'c',
                '2', 'rdamedia',
            ]))
    record.add_field(
        Field(
            tag = '338',
            indicators = [' ', ' '],
            subfields = [
                'a', 'online resource',
                'b', 'cr',
                '2', 'rdacarrier',
            ]))
    record.add_field(
        Field(
            tag = '490',
            indicators = ['1', ' '],
            subfields = [
                'a', f'[{document_type_normal}] / [Louisiana State University] ; ',
                'v', f'[{ publication_date }]',
            ]))
    record.add_field(
        Field(
            tag = '538',
            indicators = [' ', ' '],
            subfields = [
                'a', 'Mode of access: World Wide Web.',
            ]))
    record.add_field(
        Field(
            tag = '500',
            indicators = [' ', ' '],
            subfields = [
                'a', f'Title from PDF title page (viewed {datetime.date.today():%B %d, %Y}).',
            ]))
    record.add_field(
        Field(
            tag = '500',
            indicators = [' ', ' '],
            subfields = [
                'a', "Includes abstract.",
            ]))
    record.add_field(
        Field(
            tag = '502',
            indicators = [' ', ' '],
            subfields = [
                'a', '',
                'b', degree_abbr(row_dataclass["degree_name"]),
                'c', 'Louisiana State University',
                'd', f'{ publication_date }.',
            ]
        ))
    record.add_field(
        Field(
            tag = '504',
            indicators = [' ', ' '],
            subfields = [
                'a', 'Includes bibliographical references.',
            ]))
    record.add_field(
        Field(
            tag = '653',
            indicators = [' ', ' '],
            subfields = [i for i in make_keyword_subfields(row_dataclass["keywords"])],
        ))
    record.add_field(
        Field(
            tag = '710',
            indicators = ['2', ' '],
            subfields = [
                'a', "Louisiana State University (Baton Rouge, La.).",
                'b', f'Department of { row_dataclass["department"] }.',
            ]))
    record.add_field(
        Field(
            tag = '830',
            indicators = [' ', '0'],
            subfields = [
                'a', f"{document_type_normal} (Louisiana State University (Baton Rouge, La.)) ;",
                'v', f'{ publication_date }.',
            ]))
    record.add_field(
        Field(
            tag = '856',
            indicators = ['4', '0'],
            subfields = [
                'u', row_dataclass["calc_url"],
            ]))
    record.add_field(
        Field(
            tag = "949",
            indicators = [' ', ' '],
            subfields = [
                'a', f'{name_959a} {publication_date} {row_dataclass["author1_lname"]}',
                'w', 'ASIS',
                'm', 'LSU-MAIN',
                'l', 'ONLINE',
                't', 'ONLINE',
                'x', f'{name_959x}',
            ]))
            
    return record

In [25]:
thesis_records = [marc_template(i) for i in rows_dataclass if i["document_type"] == 'thesis']
diss_records = [marc_template(i) for i in rows_dataclass if i["document_type"] == 'dissertation']
maj_pap_records = [marc_template(i) for i in rows_dataclass if i["document_type"] == 'major_paper']
restricted_theses_records = [marc_template(i) for i in rows_dataclass if i["document_type"] == 'thesis_restricted']
restricted_diss_records = [marc_template(i) for i in rows_dataclass if i["document_type"] == 'dissertation_restricted']

2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2019
2020
2021
2021
2020
2021
2021
2021
2021
2021
2021
2022
2022
2021
2019
2020
2020
2021
2021
2021
2021
2021
2020
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2020
2020
2020
2021
2022
2020
2021
2021
2021
2019
2019
2019
2019
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2020
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2020
2021
2021
2021
2021
2021
2021
2022
2022
2022
2022
2019
2019
2020
2020
2020
2020
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2021
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022
2022


In [26]:
for i in (thesis_records, diss_records, maj_pap_records, restricted_theses_records, restricted_diss_records):
    print(len(i))

423
675
0
0
0


In [27]:
type_outputname = {'theses.mrc': thesis_records, 
                  'dissertations.mrc': diss_records,
                  'restricted_dissertations.mrc': restricted_diss_records,
                  'restricted_theses.mrc': restricted_theses_records}

In [28]:
for filename, dataset in type_outputname.items():
    os.makedirs('output', exist_ok=True)
    with open(os.path.join('output', filename), 'wb') as f:
        for count, record in enumerate(dataset):
    #         if count > 5:
    #             break
            f.write(record.as_marc())